In [2]:
import pandas as pd
import numpy as np
import sqlite3

# Loading Data

In [3]:
conn = sqlite3.connect("data/chinook.db")

In [4]:
query = "SELECT * FROM invoices" # select (columns) from (table)

a = pd.read_sql_query(query, conn)
a.head(4)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91


In [5]:
a['InvoiceId'].shape

(412,)

# Join Data

**Do at least 1 of the following operations to get 2**
- [x] Take data from joining minimum of 4 table 

In [6]:
# Taking data from joining 5 tables
query = """
SELECT i.InvoiceId,Country,BillingCity,tracks.Name as Tracks,Milliseconds,Bytes,genres.Name as Genre,InvoiceDate,Total
FROM customers
LEFT JOIN invoices as i ON i.CustomerId = customers.CustomerId
LEFT JOIN invoice_items as ii ON ii.InvoiceLineId = i.InvoiceId
LEFT JOIN tracks ON tracks.trackid = ii.trackid
LEFT JOIN genres ON genres.genreid = tracks.genreid
"""
#LEFT JOIN genres ON genres.genreid = tracks.genreid
data = pd.read_sql_query(query, conn)
data.head(5)

,InvoiceId,Country,BillingCity,Tracks,Milliseconds,Bytes,Genre,InvoiceDate,Total
0,98,Brazil,São José dos Campos,União Da Ilha,330945,11100945,Latin,2010-03-11 00:00:00,3.98
1,121,Brazil,São José dos Campos,Born To Move,342804,11260814,Rock,2010-06-13 00:00:00,3.96
2,143,Brazil,São José dos Campos,Esquinas,280999,9096726,Latin,2010-09-15 00:00:00,5.94
3,195,Brazil,São José dos Campos,Don't Damn Me,318901,5385886,Rock,2011-05-06 00:00:00,0.99
4,316,Brazil,São José dos Campos,All Within My Hands,527986,17162741,Metal,2012-10-27 00:00:00,1.98


In [9]:
print(data['Country'].astype('category'))

0      Brazil
1      Brazil
2      Brazil
3      Brazil
4      Brazil
        ...  
407     India
408     India
409     India
410     India
411     India
Name: Country, Length: 412, dtype: category
Categories (24, object): ['Argentina', 'Australia', 'Austria', 'Belgium', ..., 'Spain', 'Sweden', 'USA', 'United Kingdom']


In [7]:
data['Genre'].astype('category')

0      Latin
1       Rock
2      Latin
3       Rock
4      Metal
       ...  
407    Latin
408     Rock
409     Rock
410    Metal
411    Latin
Name: Genre, Length: 412, dtype: category
Categories (17, object): ['Alternative & Punk', 'Blues', 'Bossa Nova', 'Easy Listening', ..., 'Rock', 'Rock And Roll', 'Soundtrack', 'World']

# Data Preprocessing

**Do at least at least 3 of the following operations to get 3**
- [x] Datetime operation 
- [x] Categorical operation 
- [x] Frequencies analysis 
- [x] Missing Value and Duplicates operation

**Do at least 2 of the following operations to get 3**
- [X] stack/unstack
- [x] melt 
- [x] groupby 

## Menghitung rata-rata umur *employee* per jabatan

In [8]:
query = "SELECT EmployeeId,BirthDate,Title FROM employees" # select (columns) from (table)
employee = pd.read_sql_query(query, conn) 
employee.head()

,EmployeeId,BirthDate,Title
0,1,1962-02-18 00:00:00,General Manager
1,2,1958-12-08 00:00:00,Sales Manager
2,3,1973-08-29 00:00:00,Sales Support Agent
3,4,1947-09-19 00:00:00,Sales Support Agent
4,5,1965-03-03 00:00:00,Sales Support Agent


In [9]:
employee['BirthDate'] = employee['BirthDate'].astype('datetime64')
this_year=2020
born_year = employee['BirthDate'].dt.year
employee['Age'] = this_year - born_year
employee.head()

,EmployeeId,BirthDate,Title,Age
0,1,1962-02-18,General Manager,58
1,2,1958-12-08,Sales Manager,62
2,3,1973-08-29,Sales Support Agent,47
3,4,1947-09-19,Sales Support Agent,73
4,5,1965-03-03,Sales Support Agent,55


### Menghitung rata-rata umur pegawai pada setiap bidang

In [10]:
employee_age_mean = employee.groupby(['Title']).mean()
employee_age_mean.head()

,EmployeeId,Age
Title,,
General Manager,1.0,58.000000
IT Manager,6.0,47.000000
IT Staff,7.5,51.000000
Sales Manager,2.0,62.000000
Sales Support Agent,4.0,58.333333


## Preprocessing dataframe DATA

In [11]:
data.dtypes

InvoiceId         int64
Country          object
BillingCity      object
Tracks           object
Milliseconds      int64
Bytes             int64
Genre            object
InvoiceDate      object
Total           float64
dtype: object

In [12]:
# Mengubah tipe data

data['Country']=data['Country'].astype('category')
data['BillingCity']=data['BillingCity'].astype('category')
data['Genre']=data['Genre'].astype('category')
data['InvoiceDate']=data['InvoiceDate'].astype('datetime64')
data.dtypes

InvoiceId                int64
Country               category
BillingCity           category
Tracks                  object
Milliseconds             int64
Bytes                    int64
Genre                 category
InvoiceDate     datetime64[ns]
Total                  float64
dtype: object

### Menghitung data banyaknya transaksi berdasarkan negara dan genre

In [13]:
#data['Year'] = data['InvoiceDate'].dt.year
#data.head
data.head()

,InvoiceId,Country,BillingCity,Tracks,Milliseconds,Bytes,Genre,InvoiceDate,Total
0,98,Brazil,São José dos Campos,União Da Ilha,330945,11100945,Latin,2010-03-11,3.98
1,121,Brazil,São José dos Campos,Born To Move,342804,11260814,Rock,2010-06-13,3.96
2,143,Brazil,São José dos Campos,Esquinas,280999,9096726,Latin,2010-09-15,5.94
3,195,Brazil,São José dos Campos,Don't Damn Me,318901,5385886,Rock,2011-05-06,0.99
4,316,Brazil,São José dos Campos,All Within My Hands,527986,17162741,Metal,2012-10-27,1.98


In [15]:
country = data.groupby(['Country','Genre']).sum().drop(['InvoiceId'], axis=1)
country.head()

Milliseconds      Bytes  Total
Country   Genre                                             
Argentina Alternative & Punk      202527.0  6587802.0   5.94
          Blues                        NaN        NaN    NaN
          Bossa Nova                   NaN        NaN    NaN
          Easy Listening               NaN        NaN    NaN
          Electronica/Dance            NaN        NaN    NaN

In [16]:
country.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 408 entries, ('Argentina', 'Alternative & Punk') to ('United Kingdom', 'World')
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Milliseconds  129 non-null    float64
 1   Bytes         129 non-null    float64
 2   Total         129 non-null    float64
dtypes: float64(3)
memory usage: 13.3 KB


In [17]:
country[['Milliseconds','Bytes','Total']] = country[['Milliseconds','Bytes','Total']].fillna(0)
country.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 408 entries, ('Argentina', 'Alternative & Punk') to ('United Kingdom', 'World')
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Milliseconds  408 non-null    float64
 1   Bytes         408 non-null    float64
 2   Total         408 non-null    float64
dtypes: float64(3)
memory usage: 13.3 KB


### Check Missing Value

In [22]:
country.shape

(408, 3)

In [23]:
#proses pengambilan data per tiap negara
country_new=country.unstack(level=0).stack(level=0)
country_new['Argentina'].head()

Genre                           
Alternative & Punk  Bytes           6587802.00
                    Milliseconds     202527.00
                    Total                 5.94
Blues               Bytes                 0.00
                    Milliseconds          0.00
Name: Argentina, dtype: float64

### Menentukkan genre apa yang paling banyak didengar di berbagai negara

In [38]:
data.head()

,InvoiceId,Country,BillingCity,Tracks,Milliseconds,Bytes,Genre,InvoiceDate,Total
0,98,Brazil,São José dos Campos,União Da Ilha,330945,11100945,Latin,2010-03-11 00:00:00,3.98
1,121,Brazil,São José dos Campos,Born To Move,342804,11260814,Rock,2010-06-13 00:00:00,3.96
2,143,Brazil,São José dos Campos,Esquinas,280999,9096726,Latin,2010-09-15 00:00:00,5.94
3,195,Brazil,São José dos Campos,Don't Damn Me,318901,5385886,Rock,2011-05-06 00:00:00,0.99
4,316,Brazil,São José dos Campos,All Within My Hands,527986,17162741,Metal,2012-10-27 00:00:00,1.98


In [55]:
gen_in_country = pd.crosstab(
    index = [data['Country'],data['Genre']],
    columns = 'Jml Penggemar',
).sort_values(by='Jml Penggemar', ascending=False)

In [56]:
gen_in_country

col_0                          Jml Penggemar
Country        Genre                        
USA            Rock                       44
Canada         Rock                       21
USA            Latin                      16
Germany        Rock                       15
Canada         Latin                      13
...                                      ...
Denmark        Reggae                      1
Czech Republic R&B/Soul                    1
               Jazz                        1
               Easy Listening              1
France         Reggae                      1

[129 rows x 1 columns]

In [47]:
gen_in_country.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 129 entries, ('Argentina', 'Alternative & Punk') to ('United Kingdom', 'Rock')
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Jml Penggemar  129 non-null    int64
dtypes: int64(1)
memory usage: 1.7+ KB
